In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import root_mean_squared_error
import json
import joblib
import pickle

In [2]:
df_product_prices_raw = pd.read_csv(os.path.join("..", "data", "product_prices_leaflets.csv"))

In [3]:
df_product_prices_raw.head()

,sku,time_key,competitor,pvp_was,discount,flag_promo,leaflet
0,2056,20240312,chain,21.70,0.0000,0,NaN
1,4435,20230621,chain,18.92,0.2754,1,weekly
2,1951,20230303,competitorA,60.58,0.3524,1,NaN
3,2135,20240605,chain,55.37,0.2509,1,weekly
4,3949,20230529,chain,51.90,0.0000,0,NaN


In [4]:
df_product_prices = df_product_prices_raw.copy()
df_product_prices['price_final'] = df_product_prices['pvp_was'] * (1 - df_product_prices['discount'])
df_product_prices = df_product_prices.drop(columns=['pvp_was', 'discount', 'flag_promo', 'leaflet']).drop_duplicates() #removing leaflet "creates" duplicates

In [5]:
df_product_prices.head()

,sku,time_key,competitor,price_final
0,2056,20240312,chain,21.700000
1,4435,20230621,chain,13.709432
2,1951,20230303,competitorA,39.231608
3,2135,20240605,chain,41.477667
4,3949,20230529,chain,51.900000


In [6]:
df_product_prices_A = df_product_prices[df_product_prices.competitor == 'competitorA']
df_product_prices_B = df_product_prices[df_product_prices.competitor == 'competitorB']

In [7]:
def fit_pred_eval(pipeline, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    pipeline.fit(X_train, y_train)
    y_pred_train = pipeline.predict(X_train)
    y_pred_test = pipeline.predict(X_test)
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mae_test = mean_absolute_error(y_test, y_pred_test)
    rmse_train = root_mean_squared_error(y_train, y_pred_train)
    rmse_test = root_mean_squared_error(y_test, y_pred_test)
    return mae_train, mae_test, rmse_train, rmse_test

A baseline that literally only uses sku and time_key without any kind of preprocessing

In [8]:
X_A = df_product_prices_A[['sku', 'time_key']]
y_A = df_product_prices_A['price_final']

In [9]:
pipeline_lr_A = make_pipeline(LinearRegression())
mae_train, mae_test, rmse_train, rmse_test = fit_pred_eval(pipeline_lr_A, X_A, y_A)
print(f"MAE train {mae_train}, test {mae_test}, RMSE train {rmse_train}, test {rmse_test}")

MAE train 26.34663052532863, test 26.380306766264095, RMSE train 52.224400208669074, test 52.44101768348399


In [10]:
X_B = df_product_prices_B[['sku', 'time_key']]
y_B = df_product_prices_B['price_final']
pipeline_lr_B = make_pipeline(LinearRegression())
mae_train, mae_test, rmse_train, rmse_test = fit_pred_eval(pipeline_lr_B, X_B, y_B)
print(f"MAE train {mae_train}, test {mae_test}, RMSE train {rmse_train}, test {rmse_test}")

MAE train 19.500478074770253, test 19.3373096499812, RMSE train 40.41653628215445, test 39.536273081182095


In [11]:
response = {"sku": "2222", "time_key": 20250101}
prediction_df = pd.DataFrame([response], columns=['sku', 'time_key']).astype(X_A.dtypes)
pvp_is_competitorA = pipeline_lr_A.predict(prediction_df)
pvp_is_competitorB = pipeline_lr_B.predict(prediction_df)

In [12]:
pvp_is_competitorA

array([50.43489593])

In [13]:
pvp_is_competitorB

array([38.35548089])

In [14]:
with open("columns.json", 'w') as in_fd:
    json.dump(['sku', 'time_key'], in_fd)

In [15]:
with open("dtypes.pickle", 'wb') as in_fd:
    pickle.dump(X_A.dtypes, in_fd)

In [16]:
joblib.dump(pipeline_lr_A, "pipelineA.pickle")
joblib.dump(pipeline_lr_B, "pipelineB.pickle");